In [1]:
# Create file directory if didn't exist
import os

input_folder = r"data"
output_folder = r"output"

try:
    os.makedirs(input_folder, exist_ok=True)
    os.makedirs(output_folder, exist_ok=True)
except OSError:
    pass

In [ ]:
from PIL import Image, ImageSequence

def IsGif(data_path:str):
    try:
        webp_image = Image.open(data_path)
        Index = 0
        for Frame in ImageSequence.Iterator(webp_image):
            Index += 1
        return Index > 1
    except:
        return False

In [ ]:
from PIL import Image

# functin to convert webp to png/gif
def convert_webp_to_png(data_path:str, output_path:str, is_gif:bool, format='png'):
    try:
        webp_image = Image.open(data_path)
        if is_gif:
            webp_image.save(output_path, format='gif', save_all=True,lossless=True, quality=100, method=6)
        else:
            png_image = webp_image.convert("RGBA")
            png_image.save(output_path, format=format,lossless=True, quality=100, method=6)

        print(f"{data_path} convert to {output_path}")
    except Exception as e:
        print(f"An error occurred while converting {data_path}: {e}")

In [ ]:
def get_path_by_name(data_folder:str, output_folder:str, file_name:str):
    image_name, ext = os.path.splitext(file_name)
    data_path = f"{data_folder}\\{file_name}"
    output_path = f"{output_folder}\\{image_name}.png"
    is_gif = IsGif(data_path)
    if is_gif:
        output_path = f"{output_folder}\\{image_name}.gif"
    return data_path, output_path, is_gif

In [ ]:
def convert_data_by_part(data_folder:str, output_folder:str, start_index:int, end_index:int, file_list):
    print(start_index, end_index)
    total = 0
    for n in range(start_index, end_index):
        file_name = file_list[n]
        data_path, output_path, is_gif = get_path_by_name(data_folder, output_folder, file_name)
        convert_webp_to_png(data_path, output_path, is_gif)
        total += 1
    print(f"total image processed = {total}")

In [53]:
import os
from PIL import Image

def get_all_image(data_folder:str):
    file_list = os.listdir(data_folder)

    image_list = []
    for n in file_list:
        image_name, ext = os.path.splitext(n)
        image = Image.open(os.path.join(data_folder, n))

        if "png" in ext:
            image.load()

            background = Image.new("RGB", image.size, (255, 255, 255))
            background.paste(image, mask=image.split()[3])
            
        image_list.append(image)
    return image_list

In [54]:
import os
import threading
import multiprocessing
import math
import time

def combine_picture(data_folder:str, output_folder:str, target:str):
    # Start time
    start_time = time.time()
    
    image_list = get_all_image(data_folder)
    final_path = os.path.join(output_folder, f"combined.{target}")
    if len(image_list) == 0:
        return
    if len(image_list) > 1:
        image_list[0].save(final_path, target.upper(), resolution=100.0, save_all=True, append_images=image_list[1:])
    else:
        image_list[0].save(final_path, target.upper(), resolution=100.0)
    
    # Calculate time
    elapsed_time = int(time.time() - start_time)
    hour = elapsed_time//3600
    minute = (elapsed_time - hour*3600)//60
    second = (elapsed_time - hour*3600 - minute*60)
    print(f"time elapsed: {hour:02d}:{minute:02d}:{second:02d}")

In [55]:
combine_picture(input_folder, output_folder, "PDF")

[<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=256x256 at 0x23C1E1B1790>]
time elapsed: 00:00:00
